In [1]:
import os
import sys
import numpy as np

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)


from Dataset.cancer_dataset_for_LS import main_generate_cancer_matrices_for_LS, load_cancer_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b

from estimator.NDIS import BasicNDISEstimator

In [2]:
file_X_path = os.path.join(project_dir, "Dataset", "cancer-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "cancer-LR-y.txt")

X = None
y = None
try:
    X, y = load_cancer_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    main_generate_cancer_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_cancer_dataset_matrices_for_LS()

B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

In [3]:
from numpy.linalg import inv as mat_inv
from analysis.commons import twoNorm
from scipy.linalg import sqrtm, eigh

M = B.T @ B
inv_M = mat_inv(M)
sqrt_inv_M = sqrtm(inv_M)

In [4]:
from analysis.commons import compute_xopt, get_w, get_neighbor_B, get_neighbor_b

index = 186
d = B.shape[1]

r = 300
xopt = compute_xopt(B, b)
w = get_w(B, b)
cov = (np.linalg.norm(w) ** 2 / r) * np.linalg.inv(B.T @ B)

neighbor_B = get_neighbor_B(index, B)
neighbor_b = get_neighbor_b(index, b)
neighbor_w = get_w(neighbor_B, neighbor_b)
neighbor_xopt = compute_xopt(neighbor_B, neighbor_b)
neighbor_cov = (np.linalg.norm(neighbor_w) ** 2 / r) * np.linalg.inv(neighbor_B.T @ neighbor_B)


In [5]:
eps = 0

In [6]:
# compute leverage and residual 
## First Expression 
from numpy.linalg import inv as mat_inv
from analysis.commons import twoNorm
from scipy.linalg import sqrtm, eigh

M = B.T @ B
inv_M = mat_inv(M)
sqrt_inv_M = sqrtm(inv_M)
v = B[index].copy().reshape(-1, 1)
u = A[index].copy().reshape(-1, 1)
pi = (v.T @ inv_M @ v).item()
qi = (u.T @ mat_inv(A.T @ A) @ u).item()

# compute residual
ei = w[index]
error = twoNorm(w)

c11 = (1 - pi) * error**2 / ((1-pi) * error**2 - ei**2)
raw_A1 = np.eye(d) - c11 * (sqrt_inv_M @ (M - v @ v.T) @ sqrt_inv_M)
A1, U1 = eigh(raw_A1)

c12 = np.sqrt(r) * ei * error / ((1-pi) * error**2 - ei**2)
b1 = (-c12 * U1.T @ (sqrt_inv_M @ (M - v @ v.T) @ inv_M @ v)).ravel()
c1 = -0.5*np.log((1- ei**2/((1-pi)*error**2))**d*(1/(1-pi))) - 0.5*r*pi*ei**2/((1-pi) * error**2 - ei**2)


In [7]:
## Second Expression 
c11 = (1 - pi) * error**2 / ((1-pi) * error**2 - ei**2)
e1 = np.eye(d)[0].reshape(-1, 1)

A1 = np.diag(c11 * pi* e1 @ e1.T + (1 - c11)*np.eye(d))
b1 = (-np.sqrt(r)*ei*error*(1-pi)*np.sqrt(pi)/((1-pi)*error**2-ei**2)*e1).ravel()
c1 = -0.5*np.log((1- ei**2/((1-pi)*error**2))**d*(1/(1-pi))) - 0.5*r*pi*ei**2/((1-pi) * error**2 - ei**2)


In [8]:
from analysis.least_square_numerical import radial_expectation_quad_adaptive_mp

lambda2 = 1 - c11
lambda1 = c11 * pi + lambda2
lambda3 = np.sqrt(r)*ei*error*(1-pi)*np.sqrt(pi)/((1-pi)*error**2-ei**2)


val = radial_expectation_quad_adaptive_mp(lambda1, lambda2, lambda3, c1+eps, d, dps=100, workers=32)
print("Adaptive piecewise estimate:", val)

Adaptive piecewise estimate: 0.2783541269515343314843103143309837305228410886267567348327549065342038612604074198302766542261653988


In [9]:
lambda2 = -(qi - pi)/(1-qi)
lambda1 = (2*pi - pi**2 - qi)/(1-qi)

lambda3 =np.sign(ei)*np.sqrt(r)*(1-pi)*np.sqrt(pi)*np.sqrt(qi-pi)/(1-qi)

c1 = (d+1)/2*np.log(1-pi) - d/2*np.log(1-qi) - r*pi/2*(qi-pi)/(1-qi)

radial_expectation_quad_adaptive_mp(lambda1, lambda2, lambda3, c1+eps, d, dps=100, workers=32)

mpf('0.278354126934794836752350384882054119474714059756504094409833357808319650359724789690391665279836683814')

In [23]:
eps = 0.65
from analysis.least_square_numerical import delta_pq_ls

val = delta_pq_ls(p=pi, q=qi, d=d, r=r, eps=eps, workers=2, sign_w=np.sign(ei), dps=80)
print(val)


0.00000000072578677356509687291644231291083335530010168615668184608314004501226015253423074


In [24]:
from LS_mechanisms.optim_LS import lev_evaluate_ALS_right
from LS_mechanisms.optim_LS import lev_evaluate_ALS_left

lev_evaluate_ALS_right(r=r, d=d, leverage=pi, aug_leverage=qi, epsilon=eps)

(np.float64(6.229194198527011e-10), 0)